In [ ]:
import pandas as pd
import prolog as pr
import numpy as np
import clustering_optimization as co

In [ ]:
dataset_intero = pd.read_csv('../Dataset/dataset_preprocessato.csv')
dataset_utenti = pd.DataFrame(columns=['NameOrig', 'NewBalanceOrig', 'Step'])

In [ ]:
from matplotlib import pyplot as plt


X = dataset_intero.drop(columns=['IsFraud','NameDest', 'NameOrig', 'Type'])  # training set
dataset_intero = co.em_clustering(X, dataset_intero)
for i in range(0, 10):
    df = dataset_intero[(dataset_intero['Cluster'] == i)]
    df['IsFraud'].hist(bins=50, figsize=(20,15))
    plt.show()

In [ ]:
for index, element in dataset_intero.iterrows():
    if element['NameOrig'] not in dataset_utenti:
        dataset_utenti.loc[len(dataset_utenti.index)] = [element['NameOrig'], element['NewBalanceOrig'], element['Step']]
    elif element['Step'] > dataset_utenti.loc[dataset_utenti[element['NameOrig']], 'Step']:
        dataset_utenti.loc[dataset_utenti[element['NameOrig']], 'NewBalanceOrig'] = element['NewBalanceOrig']

    if element['NameDest'] not in dataset_utenti:
        dataset_utenti.loc[len(dataset_utenti.index)] = [element['NameDest'], element['NewBalanceDest'], element['Step']]
    elif element['Step'] > dataset_utenti.loc[dataset_utenti[element['NameDest']], 'Step']:
        dataset_utenti.loc[dataset_utenti[element['NameOrig']], 'NewBalanceOrig'] = element['NewBalanceOrig']

dataset_utenti.to_csv('../Dataset/dataset_utenti.csv',index=False)  

In [ ]:
# dimostra la presenza o meno di duplicati, che sarebbe un errore
'''dataset_utenti = pd.read_csv('../dataset_utenti.csv')
dataset_utenti = dataset_utenti['NameOrig']
print(dataset_utenti.duplicated())'''

In [ ]:
prolog_facts = []
for index, row in dataset_utenti.iterrows():
    # Crea un fatto Prolog come stringa
    fact = [f'utente({row['NameOrig'].lower()}, {row['NewBalanceOrig']}).']
    prolog_facts.extend(fact)
id = 0
for index, row in dataset_intero.iterrows():
    # Crea un fatto Prolog come stringa
    fact = [f"transazione({id},{row['NameOrig'].lower()}, {row['NameDest'].lower()}, {row['Type'].lower()}, {row['Amount']}, {row['Step']}, {row['Cluster']}, {row['IsFraud']})."]
    prolog_facts.extend(fact)
    id = id+1

# Scrivi i fatti Prolog in un file
with open('../Dataset/analisi_transazioni.pl', 'w') as file:
    for fact in prolog_facts:
        file.write(fact + '\n')

In [ ]:
dataset_intero['Cluster'].hist(bins=50, figsize=(20,15))
plt.show()